<a href="https://colab.research.google.com/github/sinajahangir/Flood-Resilience-Analysis/blob/main/FloodAgent_SoVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This AI agent code is designed to extract location parameters (coordinates) from a user prompt using natural language processing. After identifying the relevant location, the agent queries a flood information system or dataset to retrieve and present current flood-related details (e.g., severity, vulnerability) specific to the extracted location.
The case study presented in this notebook is associated with City of Calgary

In [2]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree # Efficient nearest neighbor search
import google.generativeai as genai
from google.colab import userdata
import os
import re # For parsing LLM output

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
os.chdir('/content/drive/MyDrive/NRC')# Change directory to where flood data is saved

In [5]:
# --- LLM Setup ---
try:
    # Using Colab Secrets for API key management
    api_key = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=api_key)
    print("Gemini API Key configured.")
    # Initialize the generative model
    llm_model = genai.GenerativeModel('gemini-1.5-flash-latest') # Use a model good at instruction following/extraction
    print(f"LLM Model '{llm_model.model_name}' initialized.")
except userdata.SecretNotFoundError:
    print("ERROR: Gemini API Key ('GEMINI_API_KEY') not found in Colab Secrets.")
    print("Please add your API key via the 'Secrets' tab (key icon) on the left.")
    llm_model = None
except Exception as e:
    print(f"An error occurred during Gemini setup: {e}")
    llm_model = None

Gemini API Key configured.
LLM Model 'models/gemini-1.5-flash-latest' initialized.


In [6]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 76.1 MB/s eta 0:00:00


In [7]:
from rasterio.warp import transform

In [18]:
class CoordinateFloodProximityAgent:
    """
    An agent that uses an LLM to extract latitude and longitude from a user prompt
    and finds the coordinates of the closest flood pixel to that location.
    Social vulnerabity data is also extracted from a csv file.
    """

    def __init__(self,
                 flood_pixel_coords: np.ndarray, sovi_coords: np.ndarray, sovi_array:np.ndarray):
        """
        Initializes the agent with building and flood location data.

        Args:
            flood_pixel_coords (np.ndarray): A NumPy array of shape (N, 2) where N
                                             is the number of flood pixels, and each
                                             row is [x, y] or [lon, lat] coordinates.
            sovi_coords (np.ndarray): A NumPy array of shape (M, 2) where M
                                      is the number of SoVI points, and each
                                      row is [x, y] or [lon, lat] coordinates.
            sovi_array (np.ndarray): A NumPy array of shape (M,) where M is the number of SoVI points.
        """
        if llm_model is None:
            raise ValueError("AI agent is not initialized. Cannot create agent.")

        self.llm = llm_model


        # Store flood pixel coordinates
        self.flood_pixels = flood_pixel_coords
        self.sovi_pixels = sovi_coords
        self.sovi_array = sovi_array
        print(f"Stored {len(self.flood_pixels)} flood pixel coordinates.")

        # Build a k-d tree for *flood pixels* for efficient nearest neighbor search
        if len(self.flood_pixels) > 0 and len(self.sovi_pixels)>0:
             print("Building k-d tree for flood pixels...")
             # Assuming flood_pixel_coords are [Lon, Lat] or [X, Y]
             self.flood_kdtree = cKDTree(self.flood_pixels)
             self.sovi_kdtree = cKDTree(self.sovi_pixels)
             print("Flood k-d tree built.")
        else:
             print("Warning: No flood pixels or SoVI was provided. Closest pixel search will not work.")
             self.flood_kdtree = None
             self.sovi_kdtree = None


    def _extract_lat_lon_from_prompt(self, prompt: str) -> tuple[float, float] | None:
        """Uses the LLM to extract latitude and longitude from the user prompt."""
        # (This function is identical to the one in the previous LocationMappingAgent)

        llm_prompt = f"""
        Analyze the following user query and extract the single pair of geographical coordinates (latitude and longitude) mentioned.
        Latitude must be between -90 and 90. Longitude must be between -180 and 180.
        Pay attention to signs (N/S, E/W) or negative values indicating direction.

        User query: "{prompt}"

        Respond ONLY with the extracted coordinates in the format:
        LATITUDE=value, LONGITUDE=value
        For example: LATITUDE=40.7128, LONGITUDE=-74.0060

        If you cannot reliably extract both a valid latitude and a valid longitude from the query, respond with the exact word "UNKNOWN".
        """
        #print(f"Sending prompt to LLM for coordinate extraction:\n---\n{llm_prompt}\n---")

        try:
            response = self.llm.generate_content(llm_prompt)
            extracted_text = response.text.strip()
            #print(f"LLM response for coordinate extraction: '{extracted_text}'")

            if extracted_text == "UNKNOWN":
                print("LLM indicated coordinates could not be found.")
                return None

            match = re.match(r"LATITUDE=(-?[\d.]+),\s*LONGITUDE=(-?[\d.]+)", extracted_text, re.IGNORECASE)

            if match:
                lat_str, lon_str = match.groups()
                try:
                    latitude = float(lat_str)
                    longitude = float(lon_str)
                    if -90 <= latitude <= 90 and -180 <= longitude <= 180:
                        # Determine UTM EPSG based on lon/lat
                        utm_zone = int((longitude + 180) / 6) + 1
                        is_northern = latitude >= 0
                        utm_epsg = 32600 + utm_zone if is_northern else 32700 + utm_zone
                        utm_crs = f"EPSG:{utm_epsg}"

                        # Convert to UTM using rasterio
                        utm_x, utm_y = transform("EPSG:4326", utm_crs, [longitude], [latitude])
                        print(f"Converted to UTM ({utm_crs}): Easting={utm_x[0]}, Northing={utm_y[0]}")

                        return utm_x[0], utm_y[0]
                    else:
                        print(f"Extracted coordinates out of valid range: Lat={latitude}, Lon={longitude}")
                        return None
                except ValueError:
                    print(f"Could not convert extracted strings to float: '{lat_str}', '{lon_str}'")
                    return None
            else:
                print(f"LLM response did not match expected format 'LATITUDE=..., LONGITUDE=...': '{extracted_text}'")
                return None

        except Exception as e:
            print(f"Error during LLM call or parsing for coordinate extraction: {e}")
            return None


    def _find_closest_flood_pixel(self, target_lat: float, target_lon: float) -> tuple[np.ndarray | None, float | None]:
        """
        Finds the nearest flood pixel using the flood k-d tree.

        Args:
            target_lat: The target latitude.
            target_lon: The target longitude.

        Returns:
            A tuple (closest_pixel_coords, distance) or (None, None) if error/no data.
            closest_pixel_coords is [longitude, latitude] or [x, y] as in the input array.
        """
        if self.flood_kdtree is None or len(self.flood_pixels) == 0:
            print("No flood pixel data or k-d tree available for search.")
            return None, None
        if self.sovi_kdtree is None or len(self.sovi_pixels) == 0:
            print("No SoVI data or k-d tree available for search.")
            return None, None

        # Ensure target coords are in the correct order for the tree ([Lon, Lat] or [X,Y])
        target_point = np.array([target_lon, target_lat])

        try:
            # Query the k-d tree: find the 1 nearest neighbor
            distance, index = self.flood_kdtree.query(target_point, k=1)
            distance_sovi, index_sovi = self.sovi_kdtree.query(target_point, k=1)
            sovi_value = self.sovi_array[index_sovi]
            closest_pixel_coords = self.flood_pixels[index]
            print(f"Closest flood pixel found: Coords={closest_pixel_coords}, Distance={distance:.4f}")
            return closest_pixel_coords, distance, sovi_value
        except Exception as e:
            print(f"Error during flood k-d tree query: {e}")
            return None, None


    def find_closest_flood_pixel_to_location(self, user_prompt: str) -> str:
        """
        Processes a user prompt to extract coordinates and find the nearest flood pixel.

        Args:
            user_prompt (str): The natural language query from the user containing coordinates.

        Returns:
            str: A natural language response summarizing the findings.
        """
        # 1. Extract Lat/Lon using LLM
        extracted_coords = self._extract_lat_lon_from_prompt(user_prompt)

        if extracted_coords is None:
            # Ask LLM to formulate a response about not finding coordinates
            try:
                 response = self.llm.generate_content(f"The user asked: '{user_prompt}'. I could not extract valid latitude and longitude coordinates from this query. Please formulate a polite response asking the user to provide clear coordinates (e.g., 'latitude 40.7, longitude -74.0').")
                 return response.text.strip()
            except Exception as e:
                 print(f"LLM failed to generate clarification response: {e}")
                 return "I couldn't identify valid geographic coordinates (latitude and longitude) in your request. Please provide them clearly."

        target_lon, target_lat = extracted_coords

        # 2. Find the closest FLOOD PIXEL computationally
        closest_pixel_coords, distance, sovi_value = self._find_closest_flood_pixel(target_lat, target_lon)

        # 3. Formulate the final response (optionally using LLM)
        if closest_pixel_coords is not None and distance is not None:
            # Assuming coords are [Lon, Lat] for reporting
            summary = (f"For the location you provided (approx. Latitude={target_lat:.6f}, Longitude={target_lon:.6f}), "
                       f"the closest flood pixel recorded in the data is at Latitude={closest_pixel_coords[1]:.6f}, Longitude={closest_pixel_coords[0]:.6f}. "
                       f"The calculated distance is about {distance:.4f} units (based on the coordinate system).")

            # Optional: Use LLM to make the response more conversational
            try:
                response_prompt = f"""
                I searched the flood data and found the closest recorded flood pixel is at coordinates Latitude={closest_pixel_coords[1]:.6f}, Longitude={closest_pixel_coords[0]:.6f}.
                The distance between the user's point and this flood pixel is {distance:.4f} meters (UTM).  The SoVI value class is {sovi_value:s} based on the NRCan social fabric product.

                Generate a concise, natural language response for the user, incorporating this information. Mention the user's coordinates, the location of the closest flood pixel, and the distance.
                If distance is greater than 30 meters, mention that the user is relatively safe. Based on the SoVI value class, give suggestions for evacuation plans.
                """
                final_response = self.llm.generate_content(response_prompt)
                return final_response.text.strip()
            except Exception as e:
                print(f"LLM failed to generate final response: {e}")
                # Fallback to the pre-formatted summary
                return summary

        elif self.flood_kdtree is None:
             # Case where no flood data was loaded
             return f"I understood the location (Lat={target_lat:.6f}, Lon={target_lon:.6f}), but I don't have any flood pixel data loaded to search against."
        else:
            # Case where search failed for other reasons
             return f"I extracted the coordinates (Lat={target_lat:.6f}, Lon={target_lon:.6f}), but encountered an error trying to find the closest flood pixel in the data."

In [13]:
df_flood=pd.read_csv('CA_202012_FLSW_U_RP100_RB_30m_4326.csv')
df_sovi=pd.read_csv('DA_SoVI_Coordinates.csv')

In [14]:
df_sovi

,Unnamed: 0,feature_id,latitude,longitude,easting,northing,utm_epsg,SoVI
0,0,0,51.137884,-114.097487,703045.534093,5.669164e+06,32611,Moderate
1,1,1,51.139760,-114.093534,703313.757959,5.669384e+06,32611,Moderate
2,2,2,51.135826,-114.095223,703212.915410,5.668942e+06,32611,Moderate
3,3,3,51.138854,-114.090465,703532.346480,5.669292e+06,32611,Moderate
4,4,4,51.139152,-114.088068,703698.721549,5.669331e+06,32611,Moderate
...,...,...,...,...,...,...,...,...
1893,1893,1893,51.041926,-113.823051,302102.951452,5.658280e+06,32612,Low
1894,1894,1894,51.029446,-113.825067,301908.400717,5.656898e+06,32612,Moderate
1895,1895,1895,51.015542,-113.831733,301381.610051,5.655370e+06,32612,Moderate
1896,1896,1896,51.060412,-113.825081,302039.521175,5.660340e+06,32612,Low


In [19]:
flood_pixels = np.array(df_flood[['easting', 'northing']])
sovi_pixels = np.array(df_sovi[['easting', 'northing']])
sovi_array = np.array(df_sovi['SoVI'])
# --- Run Example IF LLM is available and data is ready ---
if llm_model and len(flood_pixels) > 0:
    # Instantiate the agent
    try:
        agent = CoordinateFloodProximityAgent(
            flood_pixel_coords=flood_pixels,# Essential: Flood pixel data
            sovi_coords=sovi_pixels,# Optional: SoVI data
            sovi_array=sovi_array,
        )

        # --- Test Queries ---
        print("\n--- Testing Queries ---")

        prompt1 = "What's the closest flood pixel to latitude 51.01, longitude -114.261?"
        print(f"\nUser Query 1: {prompt1}")
        response1 = agent.find_closest_flood_pixel_to_location(prompt1)
        print(f"Agent Response 1:\n{response1}")

        print("\n--- End Queries ---")

    except Exception as e:
        print(f"\nAn error occurred during agent instantiation or testing: {e}")

elif not llm_model:
     print("\nSkipping example usage: LLM model not initialized.")
else:
     print("\nSkipping example usage: Placeholder data (buildings or flood pixels) is empty or missing.")

Stored 191478 flood pixel coordinates.
Building k-d tree for flood pixels...
Flood k-d tree built.

--- Testing Queries ---

User Query 1: What's the closest flood pixel to latitude 51.01, longitude -114.261?
Converted to UTM (EPSG:32611): Easting=692138.0233209993, Northing=5654507.726856011
Closest flood pixel found: Coords=[ 692178.85337866 5654494.23690821], Distance=43.0008
Agent Response 1:
Based on your coordinates and our flood data, the nearest recorded flood pixel is located at Latitude 5654494.236908, Longitude 692178.853379, approximately 43 meters away.  While this is relatively safe, given the distance is greater than 30 meters, it's still advisable to monitor the situation closely.  Because your location has a low Social Vulnerability Index (SoVI) score, you likely have access to resources and support making self-evacuation easier.  However, you should still prepare a simple evacuation plan, including identifying a safe location, pre-packing an emergency bag with essenti